<a href="https://colab.research.google.com/github/IlyaDenisov88/dataenj/blob/main/PySpark/Repartition_and_Coalesce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=af6aec987a7d445f770cd7e152aa7a116dd9faf182970eb6bb0f57b013d13eea
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


#Для чего нужен repartition?

* **Балансировка нагрузки:** Когда данные распределены неравномерно по разделам, некоторые задачи могут занять больше времени на выполнение. repartition помогает перераспределить данные более равномерно.
* **Оптимизация выполнения:** Некоторые операции, такие как join и groupBy, могут быть более эффективными, если данные предварительно перераспределены.
* **Увеличение или уменьшение количества разделов:** repartition позволяет изменять количество разделов для более эффективного использования ресурсов кластера.

Рассмотрим пример, где мы создаем DataFrame, применяем repartition, и видим, как изменяется количество разделов.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Создаем SparkSession
spark = SparkSession.builder \
    .appName("Repartition Example") \
    .getOrCreate()

# Пример данных
data = [(i, f"Name_{i % 5}") for i in range(100)]
df = spark.createDataFrame(data, ["id", "name"])

# Проверяем начальное количество разделов
initial_partitions = df.rdd.getNumPartitions()
print(f"Initial number of partitions: {initial_partitions}")

# Применяем repartition для увеличения количества разделов до 15
df_repartitioned = df.repartition(15)

# Проверяем новое количество разделов
new_partitions = df_repartitioned.rdd.getNumPartitions()
print(f"New number of partitions: {new_partitions}")

# Пример использования для улучшения выполнения join
other_data = [(i, f"OtherName_{i % 5}") for i in range(100)]
other_df = spark.createDataFrame(other_data, ["id", "other_name"])

# Применяем repartition перед join
df_repartitioned = df.repartition(10, "id")
joined_df = df_repartitioned.join(other_df, "id")

# Проверяем количество разделов после join
joined_partitions = joined_df.rdd.getNumPartitions()
print(f"Number of partitions after join: {joined_partitions}")



Initial number of partitions: 2
New number of partitions: 15
Number of partitions after join: 200


1. Создаем объект SparkSession.
2. Создаем DataFrame из примера данных.
3. Используем getNumPartitions, чтобы узнать количество разделов в исходном DataFrame.
4. Увеличиваем количество разделов до 15.
5. Проверяем количество разделов после применения repartition.
6. Создаем другой DataFrame и применяем repartition по столбцу id перед выполнением join операции. Это помогает улучшить производительность join, так как данные будут предварительно распределены.
7. Проверяем количество разделов в результирующем DataFrame после join.
8. Останавливаем SparkSession.

Если читаем с CSV какого - нибудь, то также можно указать, сколько партиций должно быть

In [3]:
# Чтение CSV-файла
df = spark.read.csv("/content/sample_data/mnist_test.csv", header=True, inferSchema=True)

# Установка количества разделов (партиций)
df = df.repartition(10)

# Проверка количества разделов
print(f"Number of partitions: {df.rdd.getNumPartitions()}")

# Останавливаем SparkSession
spark.stop()


Number of partitions: 10


# Shuffle

Партиции в процессе shuffle играют ключевую роль в распределении данных по узлам кластера. Каждая операция shuffle создает новые партиции, и количество этих партиций может быть настроено для оптимизации производительности.

Рассмотрим пример, где мы создаем DataFrame, выполняем операцию join, которая требует shuffle, и настроим количество партиций для операции shuffle.

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Создаем SparkSession и настраиваем количество партиций для shuffle
spark = SparkSession.builder \
    .appName("Shuffle Example") \
    .config("spark.sql.shuffle.partitions", "50") \
    .getOrCreate()

# Проверяем текущую настройку количества партиций для shuffle
print(f"Current shuffle partitions setting: {spark.conf.get('spark.sql.shuffle.partitions')}")

# Пример данных с большим объемом
data1 = [(i, f"Name_{i % 5}") for i in range(1000000)]
data2 = [(i, f"Category_{i % 3}") for i in range(1000000)]

df1 = spark.createDataFrame(data1, ["id", "name"])
df2 = spark.createDataFrame(data2, ["id", "category"])

# Проверяем начальное количество партиций
initial_partitions_df1 = df1.rdd.getNumPartitions()
initial_partitions_df2 = df2.rdd.getNumPartitions()
print(f"Initial number of partitions in df1: {initial_partitions_df1}")
print(f"Initial number of partitions in df2: {initial_partitions_df2}")

# Принудительно увеличиваем количество партиций перед join
df1_repartitioned = df1.repartition(100)
df2_repartitioned = df2.repartition(100)

# Проверяем количество партиций после repartition
repartitioned_partitions_df1 = df1_repartitioned.rdd.getNumPartitions()
repartitioned_partitions_df2 = df2_repartitioned.rdd.getNumPartitions()
print(f"Number of partitions in df1 after repartition: {repartitioned_partitions_df1}")
print(f"Number of partitions in df2 after repartition: {repartitioned_partitions_df2}")

# Выполняем операцию join, требующую shuffle
joined_df = df1_repartitioned.join(df2_repartitioned, "id")

# Проверяем количество партиций после join
joined_partitions = joined_df.rdd.getNumPartitions()
print(f"Number of partitions after join: {joined_partitions}")

# Останавливаем SparkSession
spark.stop()

Current shuffle partitions setting: 50
Initial number of partitions in df1: 2
Initial number of partitions in df2: 2
Number of partitions in df1 after repartition: 100
Number of partitions in df2 after repartition: 100
Number of partitions after join: 2


Параметр `spark.sql.shuffle.partitions` позволяет настроить количество партиций для операций shuffle. По умолчанию это значение равно 200, но его можно изменить в зависимости от объема данных и конфигурации кластера.

# Coalesce

coalesce оптимизирована для уменьшения количества партиций без shuffle, если это возможно.

Опа! То есть repartition всегда нужно использовать, если мы перераспределяем данные - запомнили.

То есть coalesce нужен для уменьшения количества партиций, чтобы избежать избыточного количества мелких партиций, что может улучшить производительность при выполнении некоторых операций. Приведем пример -

In [10]:
from pyspark.sql import SparkSession

# Создаем SparkSession
spark = SparkSession.builder \
    .appName("Coalesce Example") \
    .getOrCreate()

# Пример данных
data = [(i, f"Name_{i % 5}") for i in range(100)]
df = spark.createDataFrame(data, ["id", "name"])

# Проверяем начальное количество партиций
initial_partitions = df.rdd.getNumPartitions()
print(f"Initial number of partitions: {initial_partitions}")

# Применяем repartition для увеличения количества партиций до 10 (для демонстрации)
df_repartitioned = df.repartition(10)
repartitioned_partitions = df_repartitioned.rdd.getNumPartitions()
print(f"Number of partitions after repartition: {repartitioned_partitions}")

# Применяем coalesce для уменьшения количества партиций до 3
df_coalesced = df_repartitioned.coalesce(3)
coalesced_partitions = df_coalesced.rdd.getNumPartitions()
print(f"Number of partitions after coalesce: {coalesced_partitions}")




Initial number of partitions: 2
Number of partitions after repartition: 10
Number of partitions after coalesce: 3


* Создаем объект SparkSession.
* Создаем DataFrame из примера данных.
* Используем getNumPartitions, чтобы узнать количество партиций в исходном DataFrame.
* Увеличиваем количество партиций до 10 для демонстрации.
* Уменьшаем количество партиций до 3 без shuffle.
* Проверяем количество партиций после применения coalesce.

Все вроде понятно... Но, а что если все таки нужно уменьшить количество партиций с shuffle, как быть? Просто добавьте флаг True, как в примере ниже. Но! Я очень не советую уменьшать партиции в моменте, лучше перезагрузить файл с нужным количество партиций, если это возможно, чем потом пытаться делить неделимое.



In [11]:
# в новых версиях PySpark так не работает, можно сделать сначала repartition, потом coalesce
df_coalesced_with_shuffle = df_repartitioned.coalesce(3, shuffle=True)
coalesced_partitions_with_shuffle = df_coalesced_with_shuffle.rdd.getNumPartitions()
print(f"Number of partitions after coalesce with shuffle: {coalesced_partitions_with_shuffle}")
# Останавливаем SparkSession
spark.stop()

TypeError: DataFrame.coalesce() got an unexpected keyword argument 'shuffle'

# Разница repartition() и partitionBy()

`partitionBy` - это метод, используемый при записи данных в файлы (например, в форматах Parquet, ORC, etc.) для указания того, как данные должны быть разделены на партиции. Он определяет логику разделения данных на основе значений одного или нескольких столбцов. И, как всегда пример -



In [12]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("PartitionBy Example").getOrCreate()


data = [(i, f"Name_{i % 5}") for i in range(100)]
df = spark.createDataFrame(data, ["id", "name"])

# Запись DataFrame с использованием partitionBy по столбцу "name".
df.write.partitionBy("name").parquet("output/path/")
spark.stop()

В данном примере данные будут разделены по столбцу "name", и каждый уникальный `name` будет сохранен в отдельной папке. Думаю, разницу Вы уже видите. Но, давайте все таки ее запишем.

* repartition : Изменяет количество партиций для распределения вычислительной нагрузки.
* partitionBy: Определяет, как данные должны быть разделены на партиции при записи на диск.
* repartition используется в контексте изменения числа партиций в DataFrame или RDD для улучшения производительности вычислений.
* partitionBy используется в контексте записи данных в файлы, чтобы структурировать данные по определенным столбцам.

То есть грубо говоря один используется перед началом работы с файлами, а другой уже делит файлы на части после работы с ними.

